# Visualizing SIR Models for COVID-19

Objectives: Look at the rate of COVID-19 growth by different regions and estimate the SIR curve.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timezone, timedelta
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from scipy import optimize
import statsmodels.api as sm
import os
import pickle
import requests

from modeling import dataproc, optimizer, sir_model

# Load Covid-19 and Census Data

In [ ]:
datastore = dataproc.DataStore()

# Utility functions

Functions that are called to plot the curve, etc.

In [ ]:
def plot_sir_model(r, i, total_model_days, df, metric, sampling_rate, name):
    """Plot the model death rates and total deaths vs actual data.
    
    Args:
        r: Array holding daily recovered population values from SIR model
        i: Array holding daily infected population values from SIR model
        total_model_days: Total number of modeled days to plot
        df: Dataframe holding metric values.
        metric: The type of metric to plot ('Cases' or 'Deaths')
        sampling_rate: Number of samples per day used to simulate the model.
        name: A name to attach to the plot.
    """
    plot_start_time = df['Date'].min().timestamp()
    plot_step_size = 24 * 60 * 60 / sampling_rate
    plot_end_time = plot_start_time + total_model_days * 24 * 60 * 60 
    plot_timestamps = np.arange(plot_start_time, plot_end_time, plot_step_size)
    plot_dates = [datetime.utcfromtimestamp(x) for x in plot_timestamps]
    print('peak date', plot_dates[np.argmax(i)])
    # Plot peak infection
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.ticklabel_format(useOffset=False)
    ax.ticklabel_format(style='plain')
    ax.plot(plot_dates[:-sampling_rate],
            (r[sampling_rate:] - r[:-sampling_rate]),
            c='g',
            label='model ' + metric + ' rate',
            linewidth=4)
    ax.plot(df['Date'].to_list()[:-1],
            (df[metric] - df[metric].shift())[1:], label='actual ' + metric + ' rate', c='r', linewidth=4)
    ax.set_title('SIR model for ' + name)
    ax.set_xlabel('Number of days')
    ax.set_ylabel('Number of individuals')
    plt.legend()
    plt.plot()
    
    # Plot recovery
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.ticklabel_format(useOffset=False)
    ax.ticklabel_format(style='plain')
    ax.plot(plot_dates, r, c='g',
            label='model ' + metric, linewidth=4)
    ax.plot(df['Date'].to_list(), df[metric], label='actual ' + metric, c='r', linewidth=4)
    ax.set_title('SIR model for ' + name)
    ax.set_xlabel('Number of days')
    ax.set_ylabel('Number of individuals')
    plt.legend()
    plt.show()

In [ ]:
# Test out a model plot
sampling_rate = 10.0 # Times per day
total_model_days = 180 # Total number of days
population = 327.2 # Population in the US in millions
infected = 0.04 # Starting population dead = cases * case fatality rate
contacts_per_day = 0.2 # Average contacts per day for people
days_to_recover = 24 # Average days to recover (Avg 10 days until symptoms + 14 days after symptoms fade)

t, s, i, r = sir_model.compute_sir(sampling_rate, total_model_days, population, infected, contacts_per_day, days_to_recover)

fig, ax = plt.subplots(figsize=(12, 8))
ax.ticklabel_format(useOffset=False)
ax.ticklabel_format(style='plain')
ax.plot(t, s, c='y', label='susceptible', linewidth=4)
ax.plot(t, i, c='r', label='infected', linewidth=4)
ax.plot(t, r, c='g', label='recovered', linewidth=4)
ax.set_title('SIR model')
ax.set_xlabel('Number of days')
ax.set_ylabel('Number of individuals (in millions)')
plt.legend()
plt.show()


In [ ]:
# Example on looking up state and county FIPS

lookup_df = datastore.county_census_df[(datastore.county_census_df.STNAME == 'Illinois')
                             & (datastore.county_census_df.CTYNAME.isin([
                                 'Jefferson County',
                                 'DeKalb County',
                                 'DuPage County',
                                 'Grundy County',
                                 'Kankakee County',
                                 'Kane County',
                                 'Kendall County',
                                 'McHenry County',
                                 'Will County',
                             ]))
]
print('state fips', lookup_df['STATE'].iloc[0])
print('county fips', lookup_df['COUNTY'].tolist())



## Modeling parameters



In [ ]:
SIMULATION_DAYS = 150 # Total number of days to simulate when plotting forecast model.
SAMPLING_RATE = 10 # Modeling time samples per day

## Regions

Some interesting areas (Name, State FIPS, County FIPS) below. Copy one of the values in the bullet points into AREA_OF_INTEREST below.
* ('US', 'NYC', 36, [5, 47, 61, 81, 85])
* ('US', 'New Orleans', 22, [51, 71, 75, 87, 89, 95, 103, 105])
* ('US', 'Detroit', 26, [87, 93, 99, 125, 147, 163])
* ('US', 'Bay Area, CA', 6, [1, 13, 41, 55, 75, 81, 85, 95, 97])
* ('US', 'Greater LA Area, CA', 6, [37, 59, 65, 71, 111])
* ('US', 'Chicago', 17, [31, 37, 43, 63, 89, 91, 93, 111, 197])

If County FIPS is empty, this will fetch stats for the whole state:
* ('US', 'California', 6, [])
* ('US', 'New York', 36, [])
* ('US', 'Michigan', 26, [])
* ('US', 'Washington', 53, [])
* ('US', 'Alabama', 1 , [])
* ('US', 'Iowa', 19, [])
* ('US', 'Texas', 48, [])
* ('US', 'Georgia', 13, [])
* ('US', 'New Jersey', 34, [])

If Country is not US, this will fetch a country's total stats:
* ('Italy', 'Italy')
* ('Spain', 'Spain')
* ('United Kingdom', 'United Kingdom')
* ('US', 'US')


In [ ]:
AREA_OF_INTEREST = ('US', 'Texas', 48, [])
MODEL_FIT_FIRST_DATE = '2019-01-01'
MODEL_FIT_LAST_DATE = '2020-05-01'  # Fit model to data before this date, reserving later dates as holdout.
METRIC = 'Deaths'

In [ ]:
if len(AREA_OF_INTEREST) <= 2:
    area_df, population = datastore.get_time_series_for_area(AREA_OF_INTEREST[0])
else:
    area_df, population = datastore.get_time_series_for_area(
        AREA_OF_INTEREST[0], AREA_OF_INTEREST[2], AREA_OF_INTEREST[3])

area_df = area_df[
    (area_df.Date >= MODEL_FIT_FIRST_DATE) & 
    (area_df.Date <= MODEL_FIT_LAST_DATE)]
area_df = area_df[area_df[METRIC] > 0]
area_df = area_df.sort_values(by=['Date']).reset_index(drop=True)
# Validate selection through plot and inspection
plt.plot(area_df['Date'], area_df[METRIC])
area_df # Check last entries (Make sure data is good first!)
print(population)

In [ ]:
# Optional: Get rid of single day outliers by using a window to limit the outlier slope
# to the second largest/smallest slope
# Rescale to keep the total number of deaths equal.

data = dataproc.convert_data_to_numpy(area_df, METRIC)
plt.yscale('log')
plt.plot(area_df['Date'], area_df[METRIC], linewidth=4, label='raw total ' + METRIC)
plt.plot(area_df['Date'], data, linewidth=4, label='smoothed total ' + METRIC)
plt.legend()
plt.show()

plt.plot(area_df['Date'].iloc[:-1],
         area_df[METRIC].iloc[1:].to_numpy() - area_df[METRIC].iloc[:-1].to_numpy(),
         linewidth=4, label='raw ' + METRIC + ' rates')
plt.plot(area_df['Date'].iloc[:-1],
         data[1:] - data[:-1],
         linewidth=4, label='smoothed ' + METRIC + ' rates')
plt.legend()

## Fitting Values to the Model

We try to find the best fit of all parameters of the model by minimizing its mean squared error (mse) from actual data points.

Note that the simple algorithm used below is randomized and not guaranteed to be optimal, but in practice, seems to converge to a near optimal solution quickly. Also, approaches such as Bayesian optimization, annealing, and other guaranteed optimal techniques take a long time to run per iteration and have occasionally stalled the notebook.

In [ ]:
# Reasonable search regions for each parameter
recovery_days = 20.0 # This is fairly constant
pop_frac_range = [0.00005, 0.005]
infection_rate_range = [0.10, 0.80]
multiplier_range = [0.01, 200.0]

best_params, best_value = optimizer.minimize(
    data, population, recovery_days,
    pop_frac_range, infection_rate_range, multiplier_range
)
print('Param', best_param)
print('MSE', best_value)

In [ ]:
#Validation plot
validation_area_df = area_df # TODO: add holdout days
validation_area_df = validation_area_df[
    (validation_area_df.Date >= MODEL_FIT_FIRST_DATE) & 
    (validation_area_df.Date <= MODEL_FIT_LAST_DATE)
]
validation_area_df = validation_area_df[validation_area_df[METRIC] > 0]
validation_area_df = validation_area_df.sort_values(by=['Date']).reset_index(drop=True)

best_pop_frac = best_params[0]
best_infection_rate = best_params[1]
best_multiplier = best_params[2]

infected = data[0] * best_multiplier
t, s, i, r = sir_model.compute_sir(
    SAMPLING_RATE,
    SIMULATION_DAYS,
    population * best_pop_frac,
    infected,
    best_infection_rate,
    days_to_recover
)

print('Steady state population fraction affected:', best_params[0])
print('Steady state number affected (e.g. total expected deaths):', best_params[0] * population)
print('Transmissions per person per day:', best_params[1])
print('Number of cases multiplier on first day estimates', best_params[2])
print('best_mse', best_value)
plot_sir_model(r, i, SIMULATION_DAYS, validation_area_df, METRIC, SAMPLING_RATE, AREA_OF_INTEREST[1])